In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 2
setup_one_gpu(GPU)

Picking GPU 2


In [2]:
import sys
import numpy as np
import os.path as osp
from tf_lab.iclr.evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs
from tf_lab.iclr.helper import stored_synthetic_samples, top_evaluation_dir

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
class_name = 'chair'

do_jsd = False
do_mmd = True
save_res = False

top_out_dir = top_evaluation_dir
# experiment_name = 'mmd_full_train'
# sample_mmd = False
# n_samples = 5
# ref_pop = 1000
# sample_pop = 1000
# mmd_loss = 'chamfer'
# special_tag = 'chamfer_mmd_all_train'
skip = ['train', 'test']

In [5]:
if do_jsd:
    boost_samples = 1
else:
    boost_samples = 3
assert(np.logical_xor(do_jsd, do_mmd))

In [6]:
# top_sample_dir = stored_synthetic_samples(class_name)[experiment_name]

In [6]:
if save_res:
    exp_name = osp.basename(top_sample_dir)
    if do_jsd:
        f_jsd = open(osp.join(top_out_dir, class_name, 'jsd_' + special_tag + exp_name + '.txt'), 'w')
            
    if do_mmd:
        f_mmd = open(osp.join(top_out_dir, class_name, 'mmd_' + special_tag + exp_name + '.txt'), 'w')
else:
    f_jsd = sys.stdout
    f_mmd = sys.stdout

In [9]:
if experiment_name.startswith('l_') and 'gan' in experiment_name:
    epochs_to_check = np.hstack([np.array([1, 5, 10]), np.arange(100, 501, 100)])
elif experiment_name.startswith('r_gan'):
    epochs_to_check = np.hstack([np.array([1, 5, 10]), np.arange(50, 2001, 50)])

NameError: name 'experiment_name' is not defined

In [9]:
epochs_to_check = [1700]

In [7]:
sample_files = []
if 'gan' in experiment_name:
    for epoch in epochs_to_check:
        sample_files.append(osp.join(top_sample_dir, 'epoch_%d.npz' % (epoch,) ))
elif 'gmm' in experiment_name:
    for f in files_in_subdirs(top_sample_dir, '.npz'):
        sample_files.append(f) 
else:
    assert False

NameError: name 'experiment_name' is not defined

In [9]:
sample_files = \
['/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/r_gan_gp/chair/rebuttal/epoch_1700.npz']


In [10]:
evaluator = Evaluator(class_name)
evaluator.load_gt_data()
if do_jsd:
    evaluator.prepare_gt_grid_variables()

In [11]:
do_mmd = True
do_jsd = False
sample_mmd = False
mmd_loss = 'emd'
do_cov = True

In [ ]:
for sample_file in sample_files:
    evaluator.prepare_sample_data(sample_file, boost_samples)
    sample_name = osp.basename(sample_file)[:-len('.npz')]
    print sample_name
    if do_jsd:
        f_jsd.write(sample_name + '\n')
        evaluator.compute_jsd(f_out=f_jsd)
        f_jsd.flush()
    if do_mmd:
        f_mmd.write(sample_name + '\n')
        evaluator.compute_mmd(loss=mmd_loss, sample_estimator=sample_mmd, n_samples=None, 
                              ref_pop_size=None, sample_pop_size=None,
                              f_out=f_mmd, skip=skip)
        
        f_mmd.flush()
    if do_cov:
        evaluator.compute_coverage(loss=mmd_loss, sample_estimator=sample_mmd, n_samples=None, 
                                   ref_pop_size=None, sample_pop_size=None,
                                   f_out=f_mmd, skip=skip)
    print

epoch_1700
epoch_1700
val 216.16647 32.86841


In [12]:
216.47 / float(2048)

0.1056982421875

In [ ]:
if save_res:
    if do_jsd:
        f_jsd.close()
    if do_mmd:
        f_mmd.close()